In [14]:
#these are the variable inputs you must input to operate the scraper
#input username and password
username = ""
password = ""
#write the path of your chrome driver here, in quotes; directory must identify the file by name
executable_path = "chromedriver.exe"
#write the path to the folder you want to download into here, in quotes
download_path = "C:\\Users\\rlitt\Downloads\\tulsa\\"

In [15]:
#importing necessary libraries
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import JavascriptException, NoSuchElementException, StaleElementReferenceException
from selenium.common.exceptions import TimeoutException, ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException, ElementNotSelectableException
from selenium.webdriver import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from IPython.display import clear_output
from selenium.common.exceptions import ElementNotInteractableException, ElementNotSelectableException
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
import time
import os
import pickle
import csv
import unicodedata
from os import path
import csv


# this function lowers the case, strips blanks and removes linebreaks
def clean(str):
    str = str.lower()
    str = str.strip()
    str = str.replace('\n', ' ')
    return str

def try_assign(xpath):
    try:
        info = clean(driver.find_element_by_xpath(xpath).text)
    except:
        info = 'none'
    return info

def get_date(string):
    if string != 'none':
        string = string[0:10]
    else:
        string = 'none'
    return string

# this function writes multiples 'none's to a given list until that list is 3 items long
def parse_party_list(list):
    try:
        list
        if len(list) >= 3:
            pass
        elif len(list) == 2:
            list.append('none')
        elif len(list) == 1:
            list.append('none')
            list.append('none')
        else:
            list = ['none', 'none', 'none']
    except:
        list = ['none', 'none', 'none']
    return list

def more_than_three(party):
    if len(party) > 3:
        output = 'yes'
        return output
    else:
        output = 'no'
        return output

def create_dataframe(database_name, header):
    if path.exists(database_name):
        print('CSV already exists.')
    else:
        with open(database_name, 'w', newline='') as outfile:
            writer = csv.writer(outfile)
            writer.writerow(header)
        print('New CSV created.')

def create_failure_log(failure_name):
    #create blank log for failures
    if path.exists(failure_name):
        print('Failure text exists.')
    else:
        with open(failure_name, 'w') as outfile:
                writer = csv.writer(outfile)
        print('Failure text created.')

def parse_string(text, str1, str2):
    try:
        parse = text.split(str1)
        parse = parse[1].split(str2)
        parse = clean(parse[0])
    except:
        parse = 'none'
    return parse

def does_string_appear(string_to_search_in, text_to_find):
    test = string_to_search_in.find(text_to_find)
    if test != -1:
        output = 'yes'
    else:
        output = 'no'
    return output

class oklahoma():

    def load_new_driver(county_input, headless='no'):
        global driver, county, short_timeout, long_timeout

        #wait to load until
        short_timeout = 5
        long_timeout = 60
        county = county_input

        #try to close an existing driver
        try:
            driver.close()
        except:
            pass

        chrome_options = Options()
        #load headless given input
        if str(headless) == 'headless':
            chrome_options.add_argument("--headless")
        else:
            pass
        chrome_options.add_argument("--window-size=1920x700")
        chrome_options.add_argument("--disable-notifications")
        chrome_options.add_argument('--no-sandbox')
        chrome_options.add_argument('--verbose')
        chrome_options.add_experimental_option("prefs", {
                "download.default_directory": download_path,
                "download.prompt_for_download": False,
                "download.directory_upgrade": True,
                "safebrowsing_for_trusted_sources_enabled": False,
                "safebrowsing.enabled": False
                })
        driver = webdriver.Chrome(options=chrome_options, executable_path=executable_path)
        driver.get('https://www.oscn.net/dockets/')
        #wait for page to load
        WebDriverWait(driver, long_timeout).until(
        EC.presence_of_element_located((By.XPATH, '//div[@id="oscn-content"]')))

    def create_case_list(start, end, year, county):
        global case_list
        case_list = list()
        prefix = "https://www.oscn.net/dockets/GetCaseInformation.aspx?db="
        suffix = "&number="
        for i in range(start, end):
            case_number = (prefix + str(county) + suffix + 'sc-' + str(year) + '-' + str(i))
            case_list.append(case_number)

    def parse_parties(second_split):
        global party_output
        plaintiff_list = list()
        defendant_list = list()

        # pull parties info and split into a list using the "v."
        try:
            parties = driver.find_element_by_xpath('//table[@class="caseStyle"]/tbody/tr/td').text.split('v.')
            # try to split plaintiff using "and"
            try:
                plaintiff = parties[0].split(second_split)
                for i in plaintiff:
                    plaintiff_list.append(clean(i.replace('Plaintiff', '').replace(',', '').replace('.', '')))
            except:
                plaintiff = 'none'

            # try to split plaintiff using "and"
            try:
                defendant = parties[1].split(second_split)
                for i in defendant:
                    defendant_list.append(clean(i.replace('Defendant', '').replace(',', '').replace('.', '')))
            except:
                defendant = 'none'
        except:
            plaintiff = 'none'
            defendant = 'none'

        # parse lists to ensure a len of 3
        parse_party_list(plaintiff_list)
        parse_party_list(defendant_list)
        plaintiff_more_than_three = more_than_three(plaintiff_list)
        defendant_more_than_three = more_than_three(defendant_list)
        party_output = [*plaintiff_list, plaintiff_more_than_three, *defendant_list, defendant_more_than_three]


    def load_daily_docket_search():
        driver.get('https://www.oscn.net/applications/oscn/report.asp?report=DailyFilings')
        #wait for page to load
        WebDriverWait(driver, long_timeout).until(
        EC.presence_of_element_located((By.XPATH, '//div[@id="oscn-content"]')))

    def date_to_search(year, subtract_days):
        global search_date
        now = datetime.now()
        current_year = int(now.strftime("%Y"))
        current_day = now.strftime("%D")
        two_digit_year = str(year)[-2:]
        if current_year == year:
            search_date = str(datetime.today() - timedelta(days=subtract_days))[0:10]
        else:
            if subtract_days == 1:
                search_date = ('12/31/' + two_digit_year)
            elif subtract_days == 2:
                search_date = ('12/30/' + two_digit_year)
            elif subtract_days == 3:
                search_date = ('12/29/' + two_digit_year)
            elif subtract_days == 4:
                search_date = ('12/28/' + two_digit_year)
            elif subtract_days == 5:
                search_date = ('12/27/' + two_digit_year)
            elif subtract_days == 6:
                search_date = ('12/26/' + two_digit_year)

    def search_last_case(search_date):
        if clean(county) == 'oklahoma':
            county_option = '8'
        elif clean(county) == 'tulsa':
            county_option = '13'
        else:
            print('Only Oklahoma and Tulsa counties are supported.')
            #return
        prefix = '//select[@name="db"]/option['
        suffix = ']'
        xpath = (prefix + county_option + suffix)
        driver.find_element_by_xpath(xpath).click()
        driver.find_element_by_xpath('//input[@name="StartDate"]').send_keys(search_date)
        driver.find_element_by_xpath('//input[@type="SUBMIT"]').click()

    def parse_sc_table():
        global last_case_number
        #find small claims table
        sc_table = driver.find_elements_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody/tr')
        #count small claims table
        sc_rows = len(sc_table)
        row_prefix = '//*[contains(text(), "SC-")]//ancestor::tbody/tr['
        row_suffix = ']/td[1]'
        row_xpath = (row_prefix + str(sc_rows) + row_suffix)
        last_case_number = driver.find_element_by_xpath(row_xpath).text
        print("The last case for " + county + " is " + last_case_number + ' from ' + search_date + '.')

    def get_last_case_number(year, county, headless = 'no'):
        global last_case_number
        oklahoma.load_new_driver(county, headless = headless)
        oklahoma.date_to_search(year, 1)
        oklahoma.load_daily_docket_search()
        oklahoma.search_last_case(search_date)
        try:
            driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
            oklahoma.parse_sc_table()
        except:
            oklahoma.date_to_search(year, 2)
            oklahoma.load_daily_docket_search()
            oklahoma.search_last_case(search_date)
            try:
                driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
                oklahoma.parse_sc_table()
            except:
                oklahoma.date_to_search(year, 3)
                oklahoma.load_daily_docket_search()
                oklahoma.search_last_case(search_date)
                try:
                    driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
                    oklahoma.parse_sc_table()
                except:
                    oklahoma.date_to_search(year, 4)
                    oklahoma.load_daily_docket_search()
                    oklahoma.search_last_case(search_date)
                    try:
                        driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
                        oklahoma.parse_sc_table()
                    except:
                        oklahoma.date_to_search(year, 5)
                        oklahoma.load_daily_docket_search()
                        oklahoma.search_last_case(search_date)
                        try:
                            driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
                            oklahoma.parse_sc_table()
                        except:
                            oklahoma.date_to_search(year, 6)
                            oklahoma.load_daily_docket_search()
                            oklahoma.search_last_case(search_date)
                            try:
                                driver.find_element_by_xpath('//*[contains(text(), "SC-")]//ancestor::tbody')
                                oklahoma.parse_sc_table()
                            except:
                                last_case_number = 'error'
        return last_case_number


    def scrape_cases(start, end, year, county, headless = ''):

        #creating file names based on start time and date
        global database_name, failure_name, failed_cases
        now = datetime.now()
        file_path = './output/'
        database_name = (file_path + county + '-' + str(start) + '-' + str(end) + '-' + str(now.strftime("%D-%H-%M")).replace('/', '-') + '.csv')
        failure_name = (file_path + county + '-' + str(start) + '-' + str(end) + '-' + str(now.strftime("%D-%H-%M")).replace('/', '-') + '-failures.txt')

        # is selected county supported?
        if clean(county) != 'tulsa' and clean(county) != 'oklahoma':
            print('Only Tulsa and Oklahoma City are currently supported.')
        else:
            failed_cases = list()
            if clean(county) == 'tulsa':
                ok_tulsa.create_files(database_name, failure_name)
            elif clean(county) == 'oklahoma':
                ok_oklahoma.create_files(database_name, failure_name)
            #load driver
            oklahoma.load_new_driver(county, headless = headless)
            #create list of urls
            oklahoma.create_case_list(start, end, year, county)



            # load, scrape, and write data for every case in the case_numbers list
            for enum, i in enumerate(case_list):
                try:
                    # report position in loop
                    remaining = (((len(case_list)) - 1) - enum)
                    remaining = str(remaining)
                    enum = str(enum + 1)
                    print('Starting ' + enum + '. ' + remaining + ' remain.')

                    #load case_details page
                    driver.get(i)
                    #wait for page to load
                    WebDriverWait(driver, long_timeout).until(
                    EC.presence_of_element_located((By.XPATH, '//div[@id="oscn-content"]')))
                    print('Case loaded.')

                    #parse information on page

                    ampersand_test = does_string_appear(driver.find_element_by_xpath('//table[@class="caseStyle"]/tbody/tr/td').text, '&')
                    if ampersand_test != -1:
                        second_string = '& '
                    else:
                        second_string = 'AND '
                    oklahoma.parse_parties(second_string)

                    if clean(county) == 'tulsa':
                        ok_tulsa.parse_attorneys()
                    if clean(county) == 'oklahoma':
                        ok_oklahoma.parse_attorneys()


                    if clean(county) == 'tulsa':
                        ok_tulsa.parse_case_details()
                        ok_tulsa.write_data(database_name)
                    if clean(county) == 'oklahoma':
                        ok_oklahoma.parse_case_details()
                        ok_oklahoma.write_data(database_name)

                    print('Finished scraping ' + i + '.')
                    if enum == str(len(case_list)):
                        clear_output(wait=False)
                        print("HOORAY! THE PROGRAM IS FINISHED. There were " + str(len(failed_cases)) + " failures.")
                    else:
                        clear_output(wait=True)
                except:
                    #if there is a failure, this will log the case number as a failure in a new list and report in the console
                    with open(failure_name, 'a', newline='') as outfile:
                        writer = csv.writer(outfile, delimiter=',')
                        writer.writerow(i)
                    failed_cases.append(i)
                    if enum == str(len(case_list)):
                        print("The final case number resulted in an error. There have been " + str(len(failed_cases)) + " error(s) total.")
                    else:
                        print("There was an error. There have been " + str(len(failed_cases)) + " error(s) so far. There are " + remaining + " loops left.")

In [17]:
class ok_tulsa():
    def parse_attorneys():
        global attorney_output
        attorney_name = list()
        attorney_bar_num = list()
        attorney_address = list()
        attorney_rep = list()

        try:
            attorney_table = driver.find_element_by_xpath('//th[contains(text(), "Attorney")]/ancestor::table')
            attorney_rows = attorney_table.find_elements(By.TAG_NAME, 'tr')


            #now running the for loop to overwrite data if attorneys exist
            for enu, z in enumerate(attorney_rows):
                row = str(enu + 1)
                if enu % 2 == 0:
                    prefix = '//th[contains(text(), "Attorney")]/ancestor::table/tbody/tr['
                    suffix_1 = ']/td'
                    suffix_2 = ']/td[2]'
                    try:
                        attorney_list = driver.find_element_by_xpath(prefix + row + suffix_1).text.splitlines()
                    except:
                        pass
                    try:
                        attorney_list_two = attorney_list[0].split(" (")
                    except:
                        pass
                    try:
                        attorney_name.append(clean(attorney_list_two[0]).replace('  ', ' '))
                    except:
                        attorney_name.append('none')
                    try:
                        attorney_bar_num.append(clean(attorney_list_two[1]).replace('bar #', '').replace(')', ''))
                    except:
                        attorney_bar_num.append('none')
                    try:
                        attorney_address.append(clean(attorney_list[1]) + ' ' + clean(attorney_list[2]))
                    except:
                        attorney_address.append('none')
                    try:
                        attorney_rep.append(clean(driver.find_element_by_xpath(prefix + row + suffix_2).text).replace(',', ''))
                    except:
                        attorney_rep.append('none')
                else:
                    pass
        except:
            attorney_name.append('none')
            attorney_bar_num.append('none')
            attorney_address.append('none')
            attorney_rep.append('none')

        parse_party_list(attorney_name)
        parse_party_list(attorney_bar_num)
        parse_party_list(attorney_address)
        parse_party_list(attorney_rep)
        attorney_more_than_three = more_than_three(attorney_name)
        attorney_output = [attorney_name[0], attorney_bar_num[0], attorney_address[0], attorney_rep[0],
                        attorney_name[1], attorney_bar_num[1], attorney_address[1], attorney_rep[1],
                        attorney_name[2], attorney_bar_num[2], attorney_address[2], attorney_rep[2],
                        attorney_more_than_three]

    # this function parses a Tulsa case details page
    def parse_case_details():

        global case_details
        case_details = list()
        case_type_list = (driver.find_element_by_xpath('//table[@class="caseStyle"]/tbody/tr/td[2]').text).split('\n')
        case_number = clean(case_type_list[0]).replace('no. ', '')
        case_type = clean(case_type_list[1]).replace('small claims: ', '').replace('(', "").replace(')', '')
        try:
            file_date = get_date(try_assign('//td[contains(text(), "Filed:")]').split('filed')[1].replace(': ', ''))
        except:
            file_date = 'none'
        try:
            closed_date = get_date(try_assign('//td[contains(text(), "Filed:")]').split('closed')[1].replace(': ', ''))
        except:
            closed_date = 'none'
        writ_to_marshall = get_date(try_assign("//*[contains(text(), 'EXECUTION ISSUED')]/ancestor::tr"))
        writ_executed = try_assign("//*[contains(text(), 'EXECUTION RETURNED')]").replace('execution returned per writ ', '')
        amount_owed = try_assign("//*[contains(text(), 'AMOUNT IN DEBT')]").replace('amount in debt of ', '').replace('$', '').replace('<....>', '')
        judge = try_assign('//th[contains(text(), "Docket")]/ancestor::table/tbody/tr/td[3]')
        ao_2020_5_order = try_assign('//*[contains(text(), "AO-2020-5")]')
        cares_act = get_date(try_assign('//*[contains(text(), "CARES ACT")]/ancestor::tr'))
        payor = clean(parse_string(try_assign('//table[@class="docketlist"]//*[contains(text(), "RECEIPT")]'), "payor:", "paid:").replace('total amount', ''))
        try:
            disposition_list = driver.find_element_by_xpath('//*[contains(text(), "Disposition Information")]/ancestor::table/tbody/tr/td[3]').text.replace('Disposed: ', '').split(', ')
            try:
                #this line looks for settlements in disposition
                driver.find_element_by_xpath('//table[contains(text(), "Settled")]')
                disposition = 'settled'
            except:
                #otherwise it pulls the official finding at the beginning of the string
                disposition = clean(disposition_list[0])
            disposition_date = clean(disposition_list[1])[0:10]
        except:
            disposition = 'none'
            disposition_date = 'none'
        disposition_note = try_assign('//nobr[contains(text(), "DISP")]//ancestor::tr/td[3]')
        plaintiff_appeared = parse_string(disposition_note, 'plaintiff', '.')
        defendant_appeared = parse_string(disposition_note, 'defendant', '.')
        try:
            possession_test = disposition_note.find('possession of the premises')
            if possession_test == -1:
                possession_test_two = disposition_note.find('possession of property')
                if possession_test_two == -1:
                    possession_of_premises = 'no'
                else:
                    possession_of_premises = 'yes'
            else:
                possession_of_premises = 'yes'
        except:
            possession_of_premises = 'no'
        court_costs_awarded = does_string_appear(disposition_note, 'court costs against the named defendant')
        possession_only = does_string_appear(disposition_note, 'possession only')
        docket_len = len(driver.find_elements_by_xpath('//th[contains(text(), "Description")]/ancestor::table/tbody/tr'))
        docket_prefix = '//th[contains(text(), "Description")]/ancestor::table/tbody/tr['
        docket_desc_suffix = ']/td[3]'
        docket_date_suffix = ']/td[1]'
        last_action = clean(driver.find_element_by_xpath(docket_prefix + str(docket_len) + docket_desc_suffix).text)
        last_action_date = clean(driver.find_element_by_xpath(docket_prefix + str(docket_len) + docket_date_suffix).text)
        now = datetime.now()
        last_scrape = now.strftime("%D-%H:%M")
        case_details = [case_number, case_type, file_date, closed_date, writ_to_marshall, writ_executed, amount_owed, judge, disposition, disposition_date, disposition_note, plaintiff_appeared, defendant_appeared, possession_of_premises, court_costs_awarded, possession_only, payor, cares_act, ao_2020_5_order, docket_len, last_action, last_action_date, last_scrape]

    # this function creates a database and failure log with county-specific header
    def create_files(database_name, failure_name):
        header = ['case_number', 'case_type', 'file_date', 'closed_date', 'writ_to_marshall', 'writ_executed', 'amount_owed', 'judge', 'disposition', 'disposition_date', 'disposition_note', 'plaintiff_appeared', 'defendant_appeared', 'possesion_of_premises', 'court_costs_awarded', 'possession_only', 'payor', 'cares_act', 'ao_2020_5_order', 'docket_len', 'last_action', 'last_action_date', 'last_scrape', 'attorney_one', 'attorney_one_bar_num', 'attorney_one_address', 'attorney_one_rep', 'attorney_two', 'attorney_two_bar_num', 'attorney_two_address', 'attorney_two_rep', 'attorney_three', 'attorney_three_bar_num', 'attorney_three_address', 'attorney_three_rep', 'attorney_more_than_three', 'plaintiff_one', 'plaintiff_two', 'plaintiff_three', 'plaintiff_more_than_three', 'defendant_one', 'defendant_two', 'defendant_three', 'defendant_more_than_three']
        create_dataframe(database_name, header)
        create_failure_log(failure_name)

    # this function writes to a given database with county-specific output
    def write_data(database_name):
        with open(database_name, 'a', newline='') as outfile:
            writer = csv.writer(outfile, delimiter=',')
            data_out = [*case_details, *attorney_output, *party_output]
            writer.writerow(data_out)

In [21]:
oklahoma.scrape_cases(1, 2, 2019, 'tulsa')

HOORAY! THE PROGRAM IS FINISHED. There were 0 failures.


In [ ]:
oklahoma.get_last_case_number(2019, 'tulsa')